In [ ]:
"""
This simple notebook demonstrates the workflow of using the TensorFlow converter.
"""
from __future__ import print_function
import numpy as np
from tensorflow.python.tools.freeze_graph import freeze_graph

import tfcoreml
import linear_mnist_train

In [ ]:
"""
Step 0: Before you run this notebook, view run the example script linear_mnist_train.py
to get a trained TensorFlow network.
This may take a few minutes.
"""
linear_mnist_train.train()

In [ ]:
"""
Step 1: "Freeze" your tensorflow model - convert your TF model into a stand-alone graph definition file
Inputs: 
(1) TensorFlow code
(2) trained weights in a checkpoint file
(3) The output tensors' name you want to use in inference
(4) [Optional] Input tensors' name to TF model
Outputs: 
(1) A frozen TensorFlow GraphDef, with trained weights frozen into it
"""

# Provide these to run freeze_graph:
# Graph definition file, stored as protobuf TEXT
graph_def_file = './model.pbtxt'
# Trained model's checkpoint name
checkpoint_file = './checkpoints/model.ckpt'
# Frozen model's output name
frozen_model_file = './frozen_model.pb'
# Output nodes. If there're multiple output ops, use comma separated string, e.g. "out1,out2".
output_node_names = 'Softmax' 


# Call freeze graph
freeze_graph(input_graph=graph_def_file,
             input_saver="",
             input_binary=False,
             input_checkpoint=checkpoint_file,
             output_node_names=output_node_names,
             restore_op_name="save/restore_all",
             filename_tensor_name="save/Const:0",
             output_graph=frozen_model_file,
             clear_devices=True,
             initializer_nodes="")


In [ ]:
"""
Step 2: Call converter
"""

# Provide these inputs in addition to inputs in Step 1
# A dictionary of input tensors' name and shape (with batch)
input_tensor_shapes = {"Placeholder:0":[1,784]} # batch size is 1
# Output CoreML model path
coreml_model_file = './model.mlmodel'
output_tensor_names = ['Softmax:0']


# Call the converter
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file, 
        mlmodel_path=coreml_model_file, 
        input_name_shape_dict=input_tensor_shapes,
        output_feature_names=output_tensor_names)


In [ ]:
"""
Step 3: Run the converted model
"""

# Provide CoreML model with a dictionary as input. Change ':0' to '__0'
# as Swift / Objective-C code generation do not allow colons in variable names
np.random.seed(100)
coreml_inputs = {'Placeholder__0': np.random.rand(1,1,784)} # (sequence_length=1,batch=1,channels=784)
coreml_output = coreml_model.predict(coreml_inputs, useCPUOnly=False)


In [ ]:
print(coreml_output)